<a href="https://colab.research.google.com/github/m-antonio22069/Bots/blob/master/enem2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importando Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [0]:
# Importando base de dados
dataset_train = pd.read_csv('train.csv')
dataset_test = pd.read_csv('test.csv')

dados = pd.DataFrame()
index = 0
for i in dataset_test.columns:
    dados.insert(loc=index, column=i, value=dataset_train[i])
    index+=1

dados.insert(loc=47, column='NU_NOTA_MT', value=dataset_train['NU_NOTA_MT'])

In [0]:
dados.head()

In [0]:
dados.isnull().sum()

In [0]:
#Tratando dados
dados = dados.drop(columns=['NU_INSCRICAO'])
dados = dados.drop(columns = ['SG_UF_RESIDENCIA'])
dados = dados.drop(columns=['TP_DEPENDENCIA_ADM_ESC'])
dados = dados.drop(columns = ['CO_PROVA_CN'])
dados = dados.drop(columns = ['CO_PROVA_CH'])
dados = dados.drop(columns = ['CO_PROVA_LC'])
dados = dados.drop(columns = ['CO_PROVA_MT'])
dados = dados.drop(columns = ['Q027'])
dados = dados.drop(columns = ['TP_ENSINO'])


In [0]:
dados['NU_NOTA_CN']=dados['NU_NOTA_CN'].fillna(0)
dados['NU_NOTA_CH']=dados['NU_NOTA_CH'].fillna(0)
dados['NU_NOTA_LC']=dados['NU_NOTA_LC'].fillna(0)
dados['NU_NOTA_MT']=dados['NU_NOTA_MT'].fillna(0)

dados['TP_STATUS_REDACAO']=dados['TP_STATUS_REDACAO'].fillna(2)

dados['NU_NOTA_COMP1']=dados['NU_NOTA_COMP1'].fillna(0)
dados['NU_NOTA_COMP2']=dados['NU_NOTA_COMP2'].fillna(0)
dados['NU_NOTA_COMP3']=dados['NU_NOTA_COMP3'].fillna(0)
dados['NU_NOTA_COMP4']=dados['NU_NOTA_COMP4'].fillna(0)
dados['NU_NOTA_COMP5']=dados['NU_NOTA_COMP5'].fillna(0)

dados['NU_NOTA_REDACAO']=dados['NU_NOTA_REDACAO'].fillna(0)

In [0]:
dados['TP_SEXO']=dados['TP_SEXO'].replace('F',0)
dados['TP_SEXO']=dados['TP_SEXO'].replace('M',1)

In [0]:
from collections import Counter

questoes = ['Q001','Q002','Q006', 'Q024','Q025','Q026', 'Q047']

for questao in questoes:
  counter = sorted(Counter(dados[questao]))
  for letra in counter:
    dados[questao]=dados[questao].replace(letra, counter.index(letra))


In [0]:
# Extraindo correlação entre colunas
correlacao = dados.corrwith(dados['NU_NOTA_MT']).iloc[:-1].to_frame()
correlacao['abs']=correlacao[0].abs()
correlacao_ordenada = correlacao.sort_values('abs',ascending=False)[0]

fig, ax = plt.subplots(figsize=(10,20))
sns.heatmap(correlacao_ordenada.to_frame(),cmap='coolwarm',annot=True, 
            vmin=1, vmax=1, ax=ax)

In [0]:
dict(correlacao_ordenada)
key = list(correlacao_ordenada.keys())

corr = correlacao_ordenada[key.index('Q001'):]

for colunm in corr.keys():
  dados = dados.drop(columns = colunm)

In [0]:
# Separando classe e variavel

variaveis = dados.iloc[:,:21].values
classe = dados.iloc[:,21].values

In [0]:
# Dividindo Base de dados

from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(variaveis,
                                                        classe,
                                                        test_size=0.2,
                                                        random_state=0)

In [0]:
# Criando piperlines
from sklearn.pipeline import Pipeline

pip_std_ForestRandom = Pipeline([('scaler',StandardScaler()),
                                 ('RandomForestRegressor', RandomForestRegressor())])

pip_min_max_ForestRandom = Pipeline([('minmax',MinMaxScaler()),
                                 ('RandomForestRegressor', RandomForestRegressor())])

pip_std_LinearRegression = Pipeline([('scaler',StandardScaler()),
                                 ('RandomForestRegressor', LinearRegression())])

pip_min_max_LinearRegression = Pipeline([('minmax',MinMaxScaler()),
                                 ('RandomForestRegressor', LinearRegression())])

**Random forest**

In [0]:
# treinando modelo Random forest std
pip_std_ForestRandom.fit(x_treino,y_treino)

In [138]:
# Retirando score
pip_std_ForestRandom.score(x_teste, y_teste)

0.9217506657293129

In [0]:
# treinando modelo Random forest min_max
pip_min_max_ForestRandom.fit(x_treino, y_treino)

In [140]:
# Retirando score
pip_min_max_ForestRandom.score(x_teste, y_teste)

0.9218411953590518

**Linear Regressor**

In [0]:
# Treinando modelo de regressão linear std
pip_std_LinearRegression.fit(x_treino, y_treino)

In [142]:
#retirando score
pip_std_LinearRegression.score(x_teste,y_teste)

0.9119410800243928

In [0]:
# Treinando modelo de regressão linear min_max
pip_min_max_LinearRegression.fit(x_treino, y_treino)

In [144]:
#retirando score
pip_min_max_LinearRegression.score(x_teste,y_teste)

0.9119410148319098

In [0]:
# Aplicando o modelo ao problema

df = pd.DataFrame()
index = 0
for i in dados.columns:
  if i != 'NU_NOTA_MT':
    df.insert(loc=index, column=i, value=dataset_test[i])
    index+=1


# Tratando dados

df['NU_NOTA_CN']=df['NU_NOTA_CN'].fillna(0)
df['NU_NOTA_CH']=df['NU_NOTA_CH'].fillna(0)
df['NU_NOTA_LC']=df['NU_NOTA_LC'].fillna(0)

df['TP_STATUS_REDACAO']=df['TP_STATUS_REDACAO'].fillna(2)

df['NU_NOTA_COMP1']=df['NU_NOTA_COMP1'].fillna(0)
df['NU_NOTA_COMP2']=df['NU_NOTA_COMP2'].fillna(0)
df['NU_NOTA_COMP3']=df['NU_NOTA_COMP3'].fillna(0)
df['NU_NOTA_COMP4']=df['NU_NOTA_COMP4'].fillna(0)
df['NU_NOTA_COMP5']=df['NU_NOTA_COMP5'].fillna(0)

df['NU_NOTA_REDACAO']=df['NU_NOTA_REDACAO'].fillna(0)

from collections import Counter

questoes = ['Q002','Q006', 'Q024','Q026', 'Q047']

for questao in questoes:
  counter = sorted(Counter(df[questao]))
  for letra in counter:
    df[questao]=df[questao].replace(letra, counter.index(letra))


previsoes = pip_min_max_ForestRandom.predict(df)

In [0]:
resposta = pd.DataFrame()
resposta.insert(loc=0, column='NU_INSCRICAO', value=dataset_test['NU_INSCRICAO'])
resposta.insert(loc=1,column='NU_NOTA_MT', value=previsoes[:])

resposta.to_csv('answer.csv', sep=',', index=False)

resp = pd.read_csv('answer.csv')    